In [ ]:
import string
from stemming.porter2 import stem
from textblob import TextBlob
import sys  
import math
from collections import Counter
import os
import operator
import re
import scipy
from string import punctuation
import numpy
from statsmodels.stats.power import TTestIndPower
import random
# fix UTF8 errors
reload(sys)  
sys.setdefaultencoding('utf8')

techList = ["startup", "entrepreneurship", "reconstruct", "wind", "talent", "prototype", "mentorship", "hydro",
"ventur", "electrif", "internship", "tech", "technic", "technolog", "incub", "batch", "internet", "lab", "manufacturer", 
"solar", "air", "renewab", "acceler", "hub", "rd", "invest", "car", "renew", "mobil", "research", "network", "onlin", "pilot", 
"msme", "mfis", "it", "e-govern", "om", "excel", "surveil", "ict", "innov", "energi", "experi", "test", "progress", "introduc"]

peopleList = ["governor", "mentorship", "angel","trader","manufacturer","mentor","entrepreneur","secretariat",
"internship","company","graduate","trainer","supplier","auditor","youth","master","member","team","exchange","coach","committee","forum",
"institut","institution","council","offici","student","collaborate","discuss","faculti","workshop","lab","centre","agency","consult",
"particip","assist","student","depend","societi","dissemin","associ","outreach","guid","person","share","competit","firm",
"supervis","author","engag"]

locationList = ["countri", "market", "counti", "citi", "govern", "sector", "region", "state", "network", "agenc", "municip", "infrastructure",
"intern", "local", "urban", "rural"]

modifierList = ["minimum", "sound","resilient","simple","solid","good","viable","proper","light","firm","minor","suitable","feasible",
"veri","full","least","small","averag","specif","potenti","onli","possibl","more","additional","main","over","basic","high","direct","limit","various","least"]
actVerbList = ["implement","plan","oper","evalu","work","build","transport","perform","integr",
"strengthen","coordin","construct","impact","audit","exercise","design","procur","establish","conduct","start"]

timeList =["year","month","week", "day","hour","time","spring","summer","fall","winter","january","february","march",
"april", "may","june","july","august","september","october","november","december", "annual", "yearly", "monthly", "weekly", "daily", "biweekly",
"last", "first", "near", "immediate", "quick", "earli", "gradual", "rapid", "initi", "futur", "regular", "second", "third", "new", "frequent"]

financeList = ["financ","cost","procur","grant","fund","financi","loan","alloc","revenue","menu","bid","spend","economi",
"equity","lend","invest","ida","capit","account","credit"]

jargonList = ["support","develop","manag","provid","improv","inform","monitor","process","requir","assess","result","effect",
"prepar","report","target","identifi","strategi","resource","key","ensure","data","measur","staff","organ","review",
"facil","issue","carry","approach","expect","indic","effici","practic","procedur","detail","respons"]

listOfWordLists = [["actVerb", actVerbList], ["modifier", modifierList], ["location", locationList], ["finance", financeList], ["people", peopleList], ["tech", techList],  ["time", timeList], ["jargon", jargonList]]
#listOfWordLists = [["time", timeWords], ["risk", riskWords], ["customer", customerWords], ["extreme", extremeWords], ["technology", technologyWords],
#["innovative", innovativeWords], ["implementation", implementationWords]]

# get sentence length, punctuation, section length, part-of-speech, etc.
def getPunctAndPOSFeatures(fileStr):
	#pos
	#https://www.garysieling.com/blog/part-of-speech-tagging-nltk-vs-stanford-nlp
	text = nltk.word_tokenize(fileStr)
	tagged = nltk.pos_tag(text)
	pos = Counter([y[1] for y in tagged])
	pos = Counter({k: v / float(len(tagged)) for k, v in pos.items()})
	#punct
	sentences = sentenceTokenizer.tokenize(fileStr)
	wordsInSentences = [len(nltk.word_tokenize(sentence)) for sentence in sentences]
	wordsPerSentence = float(sum(wordsInSentences))/len(wordsInSentences)
	wordsInSection = sum(wordsInSentences)
	punctuation_counts =  {k:v/float(sum(wordsInSentences)) for k, v in Counter(fileStr).iteritems() if k in punctuation}
	return wordsPerSentence, wordsInSection, punctuation_counts, pos

def countAbbrevs(fileStr):
	#tokenize words
	words = fileStr.split()
	#find all caps words
	abbrevs = [x for x in words if x.isupper() and not any(char.isdigit() for char in x) and len(x) > 1]
	#num abbrevs, num unique abbrevs
	return len(abbrevs), len(set(abbrevs))


# count frequencies of different categories of words (time-based, risk-based) and compute sentiment scores
def countWords(fileStr):
	#remove punctuation
	fileStr = fileStr.translate(None, string.punctuation)
	#lowercase all
	fileStr = fileStr.lower()
	#get rid of troublesome characters
	fileStr = unicode(fileStr, errors='ignore')
	####print TextBlob(fileStr).sentiment
	#tokenize words
	myDict = fileStr.split()
	#stem words
	myDict = [stem(word) for word in myDict]
	allCatsFreqs = {}
	#stem words in matching wordList
	for category in listOfWordLists:
		catWordFreqs = {}
		wordList = [stem(word) for word in category[1]]
		for word in wordList:
			catWordFreqs[word] = float(myDict.count(word)) / len(myDict)
		#print catWordFreqs
		allCatsFreqs[category[0]] = catWordFreqs
	return allCatsFreqs

def countWordsAndCompareToBaseline(fileStr, fileType, baselines):
	#tokenize words
	myDict = fileStr.split()
	#stem words
	myDict = [stem(word) for word in myDict]
	allCatsFreqs = {}
	#stem words in matching wordList
	for category in listOfWordLists:
		catWordFreqs = {}
		wordList = [stem(word) for word in category[1]]
		for word in wordList:
			if fileType in baselines[category[0]][word]:
				baselineWord = baselines[category[0]][word][fileType]
				if baselineWord != 0.0:
					catWordFreqs[word] = (float(myDict.count(word)) / len(myDict)) / baselineWord
			#catWordFreqs[word] = (float(myDict.count(word)) / len(myDict))
		#print fileType, category[0], statistics.mean(catWordFreqs) if len(catWordFreqs) > 0 else None
		allCatsFreqs[category[0]] = catWordFreqs
	return allCatsFreqs

def safeCheck(dic, word):
	return dic[word] if word in dic else 0.0

def getFeaturesForModel(features):
	return [[x['numUniqueAbbrevs'], x['numAbbrevs'], x['sentiment'].polarity, x['sentiment'].subjectivity, x['wordsPerSentence'], x['wordsInSection'], safeCheck(x['punctuationCounts'], '$'), 
	safeCheck(x['punctuationCounts'], ')'), safeCheck(x['punctuationCounts'], '-'),safeCheck(x['punctuationCounts'], ','), 
	safeCheck(x['punctuationCounts'], ':'), x['actVerb'], x['modifier'], x['location'], x['finance'], x['people'], x['tech'], x['time'], x['jargon'],
	(safeCheck(x['pos'], 'NN') + safeCheck(x['pos'], 'NNP') + safeCheck(x['pos'], 'NNS')), safeCheck(x['pos'], 'IN'), safeCheck(x['pos'], 'JJ'),
	safeCheck(x['pos'], 'DT'), safeCheck(x['pos'], 'CC'), safeCheck(x['pos'], 'MD'), safeCheck(x['pos'], 'RB'), 
	(safeCheck(x['pos'], 'VB') + safeCheck(x['pos'], 'VBN') + safeCheck(x['pos'], 'VBG') + safeCheck(x['pos'], 'VBZ') + safeCheck(x['pos'], 'VBP')),
	safeCheck(x['pos'], 'CD'), safeCheck(x['pos'], 'TO')] for x in features]

def main(innovPath, nonInnovPath):
	pcodes = [line.rstrip('\n').split('\t') for line in open('pcodes.txt')]
	cats = [x[0] for x in listOfWordLists]

	# get baseline
	baselineDicts = transformDocToDict("FULL/components/non_innov_non_innov_TTL/baseline", None, countWords)
	baselineDictsWithFileTypes = [[projInfo[7].lower()] + processed for processed in baselineDicts for projInfo in pcodes if processed[0]==projInfo[0]]
	baselines = computeAvgWordFreqOverDict(baselineDictsWithFileTypes)

	### EVALUATE DICTIONARIES
	print "INNOV"
	evaluate_innov = evaluateDocs("FULL/components/innov/", pcodes, baselines)
	innov_long = [x for x in evaluate_innov if x['fileType'] == "long" ]
		
	#non innov projs from non innov ttls!
	print "NON-INNOV"
	evaluate_non_innov = evaluateDocs("FULL/components/non_innov_non_innov_TTL/test", pcodes, baselines)
	non_innov_long = [x for x in evaluate_non_innov if x['fileType'] == "long" ]

	from sklearn.linear_model import LogisticRegression

	long_features_train = getFeaturesForModel(innov_long[0:int(0.5*len(innov_long))]) + getFeaturesForModel(non_innov_long[0:int(0.5*len(non_innov_long))])
	long_features_test = getFeaturesForModel(innov_long[int(0.5*len(innov_long)):]) + getFeaturesForModel(non_innov_long[int(0.5*len(non_innov_long)):])

	long_labels_train = [1.0] * int(0.5*len(innov_long)) + [0.0] * int(0.5*len(non_innov_long))
	long_labels_test = [1.0] * (len(innov_long) - int(0.5*len(innov_long))) + [0.0] * (len(non_innov_long) - int(0.5*len(non_innov_long)))
	model = LogisticRegression()
	model = model.fit(long_features_train, long_labels_train)
	prediction_lr = model.predict(long_features_test)
	print(metrics.classification_report(long_labels_test, prediction_lr))
	metrics.accuracy_score(prediction_lr, long_labels_test)
	# accuracy is 0.68

	# compare innov with non-innov
	# long only
	for category in cats:
		print category
		innovMeans = [statistics.mean(x[category]) for x in innov_long]
		cappedInnovMeans = removeAndCapOutliers(innovMeans)
		nonInnovMeans = [statistics.mean(x[category]) for x in non_innov_long]
		cappedNonInnovMeans = removeAndCapOutliers(nonInnovMeans)
		print cappedInnovMeans, cappedNonInnovMeans
		print statistics.mean(cappedInnovMeans), statistics.mean(cappedNonInnovMeans)
		print scipy.stats.ttest_ind(cappedInnovMeans, cappedNonInnovMeans, axis = 0, equal_var=False)
		print "\n \n \n"

	####non innov projs from innov TTLs
	print "NON-INNOV - innov ttls"
	evaluate_non_innov_innov_ttl = transformDocToDict("FULL/components/non_innov_innov_TTL/", None, countWordsAndCompareToBaseline)
	for category in cats:
		docMeans = [statistics.mean(x[category]) for x in evaluate_non_innov_innov_ttl]
		print category
		print docMeans
		print statistics.mean(docMeans)

def transformDocToDict(inputDir, extractFn = None, processFn = None):
	listOfDicts = []
	listOfFailedExtractions = []
	for filename in os.listdir(inputDir):
		print filename
		f = open(inputDir + "/" + filename, 'r')
		fileStr =  f.read().lower()
		# find detailed project description
		# find last instance of desired annex
		if (extractFn != None):
			startInd, endInd = extractFn(fileStr)
			if startInd < 0 or endInd < 0 or (endInd - startInd) < 500:
				print filename, "not found!"
				listOfFailedExtractions.append(filename)
				continue
			section = fileStr[startInd:endInd]
			if "........" in section: 
				print filename, "not found!"
				listOfFailedExtractions.append(filename)
				continue
			else:
				print filename, "found!"
		else:
			section = fileStr
		docDict = processFn(section)
		listOfDicts.append([filename[0:7], docDict])
	return listOfDicts

def evaluateDocs(inputDir, pcodes, baselines, getDictFeatures = True):
	docFeatures = []
	sentenceTokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
	for filename in os.listdir(inputDir):
		features = {'name': filename[0:7]}
		print filename
		f = open(inputDir + "/" + filename, 'r')
		fileStr =  f.read()
		#get rid of troublesome characters
		fileStr = unicode(fileStr, errors='ignore')
		#
		#get punct features
		wordsPerSentence, wordsInSection, punctuation_counts, pos = getPunctAndPOSFeatures(fileStr)
		features['wordsPerSentence'] = wordsPerSentence
		features['wordsInSection'] = wordsInSection
		features['punctuationCounts'] = punctuation_counts
		features['pos'] = pos
		#
		#remove punctuation
		exclude = set(string.punctuation)
		fileStr = ''.join(ch for ch in fileStr if ch not in exclude)
		#
		# count abbrevs
		numAbbrevs, numUniqueAbbrevs = countAbbrevs(fileStr)
		features['numAbbrevs'] = numAbbrevs
		features['numUniqueAbbrevs'] = numUniqueAbbrevs
		#
		# lower case 
		fileStr = fileStr.lower()
		# get sentiment
		features['sentiment']  = TextBlob(fileStr).sentiment
		#
		fileType = [x[7] for x in pcodes if x[0] == filename[0:7]][0].lower()
		if getDictFeatures:
			if fileType == "pad" or fileType == "project paper":
				features['fileType'] = "long"
				features['dictFeatures'] = countWordsAndCompareToBaseline(fileStr, "long", baselines)
			else:
				features['fileType'] = "short"
				features['dictFeatures'] = countWordsAndCompareToBaseline(fileStr, "short", baselines)
		for cat, catDict in features['dictFeatures'].iteritems():
			#print catDict
			if len(catDict.values()) > 0:
				catMean = sum(catDict.values())/float(len(catDict.values()))
				features[cat] = catMean
		docFeatures.append(features)
	return docFeatures

#use this method to compute baselines for each category, filetype pair (e.g. 'time', "LONG")
#filetype options are LONG, SHORT
#PAD includes PAD and proj paper
#non-PAD includes PID, PCN, and misc.
def computeAvgWordFreqOverDict(listOfDicts):
	dict0 = listOfDicts[0][2]
	allCatMeans = {}
	for k, v in dict0.iteritems():
		oneCatMean = {}
		allDictsForCat = [[x[2][k], x[0]] for x in listOfDicts]
		longWordMeans = []
		shortWordMeans = []
		for kWord, vWord in allDictsForCat[0][0].iteritems():
			for currDict in allDictsForCat:
				if currDict[1] == "pad" or currDict[1] == "project paper":
					longWordMeans.append(currDict[0][kWord])
				else:
					shortWordMeans.append(currDict[0][kWord])
			meanLongWordFreq = sum(longWordMeans)/len(longWordMeans)
			#meanShortWordFreq = sum(shortWordMeans)/len(shortWordMeans)
			oneCatMean[kWord] = {'long': meanLongWordFreq}
		allCatMeans[k] = oneCatMean
	return allCatMeans

#postprocessing
def removeAndCapOutliers(dataList):
	mean = statistics.mean(dataList)
	sd = statistics.stdev(dataList)
	upperCutoff = mean + 3 * sd
	lowerCutoff = mean - 3*sd
	print upperCutoff, lowerCutoff
	print [x for x in dataList if x > upperCutoff or x < lowerCutoff]
	finalList = []
	for x in dataList:
		if x > upperCutoff:
			finalList.append(upperCutoff)
		elif x < lowerCutoff:
			finalList.append(lowerCutofff)
		else:
			finalList.append(x)
	return finalList


### THE FOLLOWING METHODS HELP GENERATE WORD CANDIDATES FOR DICTS
def combinedDicts(listOfDicts):
	print "combining document dictionaries"
	fullDict = {}
	for currDict in listOfDicts:
		for k, v in currDict.iteritems():
			if k in fullDict:
				fullDict[k].append(v)
			else:
				fullDict[k] = [v]
	fullDict = {k: v + [0.0] * (len(listOfDicts) - len(v)) for k, v in fullDict.items()}
	return fullDict
	
	# remove words in less than N docs
	# repeat for set of non-innovative projeccts
def compareDicts(innovDict, nonInnovDict):
	# do t-tests to compare freqs of of any word that appears at least once in innov Dict
	# get # of docs each corpus
	print "comparing full dictionaries"
	numDocsNonInnov = len(nonInnovDict['the'])
	numDocsInnov = len(innovDict['the'])
	compareDictResults = {}
	powerCalc = TTestIndPower()
	for k, v in innovDict.iteritems():
		innovFreqs = innovDict[k]
		if k in nonInnovDict:
			nonInnovFreqs = nonInnovDict[k]
		else:
			nonInnovFreqs = [0.0] * numDocsNonInnov
		# get max freqs
		maxInnovFreq = max(innovFreqs)
		maxNonInnovFreq = max(nonInnovFreqs)
		# t test, 2 sample unequal variance
		t_test = scipy.stats.ttest_ind(innovFreqs, nonInnovFreqs, axis=0, equal_var=False)
		# power calculation, 2 sample t test with pooled std. get minimal detectable effect
		mde = powerCalc.solve_power(effect_size=None, nobs1=numDocsInnov, alpha=0.05, power=0.8, ratio= float(numDocsNonInnov)/numDocsInnov, alternative='two-sided') * pooledStd(innovFreqs, nonInnovFreqs)
		compareDictResults[k] = {'meanInnovWordFreq': sum(innovFreqs) / len(innovFreqs), 'fracOfInnovDocsAppearing': sum(x > 0 for x in innovFreqs) / (0.0 + numDocsInnov), 
								'meanNonInnovWordFreq': sum(nonInnovFreqs) / len(nonInnovFreqs), 'fracOfNonInnovDocsAppearing': sum(x > 0 for x in nonInnovFreqs) / (0.0 + numDocsNonInnov),
								'innovMinusNonInnovFreq': sum(innovFreqs) / len(innovFreqs) - sum(nonInnovFreqs) / len(nonInnovFreqs),
								'p_val': t_test[1], 'mde': mde, 'maxInnovFreq': maxInnovFreq, 'maxNonInnovFreq': maxNonInnovFreq, 'maxInnovMinusNonInnovFreq': maxInnovFreq - maxNonInnovFreq}
	sorted_compare = sorted(compareDictResults.items(), key=lambda t: -1 * t[1]['meanInnovWordFreq'])
	return sorted_compare

def pooledStd(arr1, arr2):
	return math.sqrt(((len(arr1) - 1) * numpy.var(arr1) + (len(arr2) - 1) * numpy.var(arr2)) / (len(arr1) + len(arr2) - 2))

def stdev(arr):
	norm = [x / sum(arr) for x in arr ]
	total = 0
	avg = 1.0/len(norm)
	for i in norm:
		total += (i - avg)**2
	total = total / len(norm)
	print math.sqrt(total)